In [1]:
%load_ext autoreload
%autoreload 2
from collections import defaultdict
import pandas as pd
import imodelsx.llm
import numpy as np
import paper_setup
import joblib
import mdcalc
import paper_parsing
import openai
import sys
sys.path.append('../notebooks_llm/')
import extract
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()
imodelsx.llm.LLM_CONFIG['LLM_REPEAT_DELAY'] = 1

### Load df with groundtruth values and paper ids

In [2]:
# paper_setup.download_open_source_papers(df) 
# need to first download papers from https://drive.google.com/drive/folders/1OUXtsddxEAOl3tKEZegBQSwArUecb-6J into ../papers
df, ids_with_paper = paper_setup.download_gsheet(assert_checks=True)

# export missing papers
# cols = ['id', 'ref_text', 'ref_href']
# UNKS = {'-1', '', None, np.nan}
# idx = (df['found_paper (0=no, 1=yes)'] == 0) * (~(df['ref_href'].isin(UNKS))) * df['ref_href'].notnull()
# df[idx][cols].to_csv('missing_papers.csv', index=False)

### Extract info from the pdfs -- add values to the following columns:
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

In [3]:
# extract text from pdfs (create file num.txt for each file num.pdf)
# paper_setup.extract_texts_from_pdf(ids_with_paper, papers_dir=paper_setup.papers_dir)
df['paper___raw_text'] = df['id'].apply(mdcalc.try_or_none(paper_parsing.get_paper_text))
texts = df[df.id.isin(ids_with_paper)]['paper___raw_text'].tolist()
extractions = extract.extract_nums_df(texts)
for k in extractions.keys():
    df.loc[df.id.isin(ids_with_paper), k] = extractions[k]

attempting to add dict_keys(['num_male', 'num_female', 'num_total', 'evidence_span_gender'])


0it [00:00, ?it/s]

542it [00:00, 1196.04it/s]


attempting to add dict_keys(['num_white', 'num_black', 'num_latino', 'num_asian', 'evidence_span_race'])


542it [00:00, 1344.14it/s]

Unterminated string starting at: line 6 column 23 (char 105)


### Final process and save

In [4]:
def int_or_neg1(x):
    try:
        return int(x)
    except:
        return -1

df = paper_parsing.check_race_keywords(df, ids_with_paper)
df["paper_contains_race_keywords"].sum()
# convert columns to int
cols_int = ["ref_year", "found_paper", "paper_contains_race_keywords"]
for c in cols_int:
    df[c] = df[c].apply(int_or_neg1)
df = df.sort_values(
    by=["found_paper", "paper_contains_race_keywords", "ref_year", "id"],
    ascending=False,
)

df.drop(columns='paper___raw_text').to_csv("../data/main.csv", index=False)

  0%|          | 0/542 [00:00<?, ?it/s]

100%|██████████| 542/542 [00:00<00:00, 1226.94it/s]
